# Web Scraping - Foodist

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-libraries" data-toc-modified-id="Importing-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing libraries</a></span></li><li><span><a href="#General-overview" data-toc-modified-id="General-overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>General overview</a></span></li><li><span><a href="#Web-Scraping" data-toc-modified-id="Web-Scraping-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Web Scraping</a></span><ul class="toc-item"><li><span><a href="#Getting-product-links" data-toc-modified-id="Getting-product-links-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Getting product links</a></span></li><li><span><a href="#Getting-all-product-infos" data-toc-modified-id="Getting-all-product-infos-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Getting all product infos</a></span><ul class="toc-item"><li><span><a href="#Function-to-get-details" data-toc-modified-id="Function-to-get-details-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Function to get details</a></span></li><li><span><a href="#Function-to-create-dataframe" data-toc-modified-id="Function-to-create-dataframe-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Function to create dataframe</a></span></li></ul></li></ul></li></ul></div>

## Importing libraries

In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## General overview

|	Product name	|	Brand	|	Price	|	Weight	|	Calories	|	Fat	|	Saturated Fats	|	Carbohydrates	|	Sugar	|	Fibre	|	Protein	|	Salt	|	Ingredients	|	Photo link	|
|	:----------	|	:----------	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|	:---------:	|
|	e.g. Snack bar	|		|	in EUR	|	in g/kg	|	in kj/kcal	|	in g	|	in g	|	in g	|	in g	|	in g	|	in g	|	in g	|		|		|

**List of websites:**
- Koro: https://www.korodrogerie.de/snacks/?p=1&o=2&n=144&f=233
- Kokku: https://kokku-online.de/vegane-suessigkeiten-snacks/
- Foodist: https://www.foodist.de/suesses-snacks?p=1&o=9&n=84&f=36

## Web Scraping

### Getting product links

**Testing**

In [9]:
# storing url in a variable
url_1 = "https://www.foodist.de/suesses-snacks?p=1&o=9&n=84&f=36"

# downloading html-code with a get request
response_1 = requests.get(url_1)

response_1.status_code

200

In [10]:
# parsing html (creating the 'soup')
soup_1 = BeautifulSoup(response_1.content, "html.parser")


In [11]:
# retrieving/extracting the desired info

# general product info
product_info = soup_1.find_all("div", class_="product--title")

# product title
product_info[14]

<div class="product--title"> KakaoWUMMS - Gebrannte Kakaobohnen mit Chili </div>

In [12]:
# general product info
product_info = soup_1.find_all("div", class_="price--unit")

# product price
product_info[0].getText()

'  3,36\xa0€ / 100 g  '

In [13]:
# general product info
product_info = soup_1.find_all("div", class_="product--box box--basic")
product_info_2 = soup_1.find_all("a", class_="product--link")

# product link
#product_info[0].find_all("a", class_="product--link") #.get("href")
product_info_2[0].get("href")

'https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685'

**Functions**

In [14]:
def get_first_details(soup):
    
    link = soup.find_all("a", class_="product--link")
    name = soup.find_all("div", class_="product--title")
    brand = soup.find_all("div", class_="product--supplier")
    price = soup.find_all("div", class_="product--price")
    price_gr = soup.find_all("div", class_="price--unit")
    
    names = []
    brands = []
    prices = []
    prices_gr = []
    links = []
    
    
    for p in name:
        name = p.getText()
        names.append(name)
        
    for p in brand:
        brand = p.getText()
        brands.append(brand)
        
    for p in price:
        price = p.getText()
        prices.append(price)
    
    for p in price_gr:
        price = p.getText()
        prices_gr.append(price)
    
    for p in link:
        link = p.get("href")
        links.append(link)
    
    return [names, brands, prices, prices_gr, links]

In [15]:
url = "https://www.foodist.de/suesses-snacks?p=1&o=9&n=84&f=36"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

products = get_first_details(soup)

In [16]:
len(products[0]), len(products[1]), len(products[2]), len(products[3]), len(products[4])

(91, 91, 84, 84, 91)

In [17]:
def scraping_pages(page):
    
    names = []
    brands = []
    links = []
    
    for p in page:
        url = "https://www.foodist.de/suesses-snacks?p={p}&o=9&n=84&f=36"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        link = soup.find_all("a", class_="product--link")
        name = soup.find_all("div", class_="product--title")
        brand = soup.find_all("div", class_="product--supplier")
        
        for p in name:
            name = p.getText()
            names.append(name)
        
        for p in brand:
            brand = p.getText()
            brands.append(brand)
    
        for p in link:
            link = p.get("href")
            links.append(link)


    return [names, brands, links]

In [18]:
products = scraping_pages(page=range(1,7))

In [19]:
len(products[0]), len(products[1]), len(products[2])

(546, 546, 546)

**Dataframe**

In [20]:
# Building a dataframe

product_overview = pd.DataFrame(
    {"product": products[0],
     "brand": products[1],
     "links": products[2]})

In [21]:
pd.set_option('display.max_rows', None)

In [62]:
product_overview.sample(20)

,product,brand,links
93,BIO Chiasamen 1kg,Foodist,https://www.foodist.de/foodist-bio-chiasamen-1...
353,Veganer Proteinriegel mit Kakao und Haselnüssen,Zebra,https://www.foodist.de/zebra-veganer-proteinri...
20,Dextrose Bonbons mit Fruchtgeschmack,frufree,https://www.foodist.de/frufree-dextrose-bonbon...
280,"Foodist Snack Bites Probierpaket, 10-teilig",Foodist,https://www.foodist.de/foodist-foodist-snack-b...
203,BIO Schokoladen Weihnachtsmann,Foodist,https://www.foodist.de/foodist-bio-schokoladen...
27,BIO Pekannusskerne,Foodist,https://www.foodist.de/foodist-bio-pekannusske...
19,Fruchtpapier Apfel & Erdbeere,Dörrwerk,https://www.foodist.de/doerrwerk-fruchtpapier-...
236,Schoko Crunchies Vollm*lch Zimt,Foodist,https://www.foodist.de/foodist-schoko-crunchie...
539,BIO Cookies mit Pistazien und Zitrone,Green & More,https://www.foodist.de/green-more-bio-cookies-...
442,"Maiskörner, geröstet & gesalzen 1kg",Foodist,https://www.foodist.de/foodist-maiskoerner-ger...


### Getting all product infos

**Testing**

In [63]:
# storing url in a variable
url = "https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854"

# downloading html-code with a get request
response = requests.get(url)
response.status_code

200

In [64]:
soup = BeautifulSoup(response.content, "html.parser")

In [65]:
# price
price = soup.find_all("div", class_="price--default")
price[0].getText()

'  7,90\xa0€ '

In [66]:
# weight
weight = soup.find_all("span", class_="product--unit")
weight[0].getText()

'  Inhalt:  125\xa0g '

In [67]:
# price/weight
price_gr = soup.find_all("span", class_="product--reference-unit-price")
price_gr[0].getText()

' 6,32\xa0€ / 100\xa0g '

In [68]:
# kcal
# nutrients
nutritients = soup.find_all("tbody")
nutritients[0].getText()

'  Brennwert 2.022 kJ(442 kcal)   Fett 26,0 g   davon gesättigte Fettsäuren 2,0 g   Kohlenhydrate 59,0 g   davon Zucker 52,0 g   Eiweiß 11,0 g   Salz 0,00 g  '

In [69]:
# photo
photo = soup.find_all("span", class_="image--media")
photo[0].canvas.get("data-image")

'https://foodist.imgix.net/media/image/5602132652727_0.jpg?auto=compress%2Cformat&h=60&q=20&w=60'

In [84]:
# link
link = soup.find_all("meta", property="og:url")
link[0].get("content")

'https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854'

In [146]:
# product name
name = soup.find_all("h1", class_="product--title")
name[0].getText()

'Mandeln mit Zimt & Zucker'

In [148]:
# brand
brand = soup.find_all("a", class_="product--supplier")
brand[0].getText()

'Vista Portuguese'

In [30]:
# price
price = soup.find_all("div", class_="price--default")

# weight
weight = soup.find_all("span", class_="product--unit")

# price/weight
price_gr = soup.find_all("span", class_="product--reference-unit-price")

# nutrients
nutritients = soup.find_all("td", class_="nutrition-value")

# ingredients
ingredients = soup.find_all("span", class_="base-info--label")

# photo
photo = soup.find_all("span", class_="image--media")

#### Function to get details

In [151]:
def scraping_details(links, *skip_link):
    
    lst_name = []
    lst_brand = []
    lst_link = []
    lst_price = []
    lst_weight = []
    lst_price_gr = []
    lst_nutrients = []
    # lst_ingredients = []
    lst_photos = []

    for l in links:
        
        if l not in skip_link:
            
            try:

                url = l
                response = requests.get(url)

                soup = BeautifulSoup(response.content, "html.parser")
                
                # product name
                name = soup.find_all("h1", class_="product--title")
                
                # brand
                brand = soup.find_all("a", class_="product--supplier")
                
                # price
                price = soup.find_all("div", class_="price--default")

                # weight
                weight = soup.find_all("span", class_="product--unit")

                # price/weight
                price_gr = soup.find_all("span", class_="product--reference-unit-price")

                # nutrients
                nutritients = soup.find_all("td", class_="nutrition-value")

                # ingredients
                # ingredients = soup.find_all("span", class_="base-info--label")

                
                # getting text for all necessary features
                for i in [name, brand, price, weight, price_gr]:
                    for j in range(len(i)):
                        i[j] = i[j].getText()

                
                # photo + getting photo link
                photo = soup.find_all("span", class_="image--media")

                photo_link = photo[0].canvas.get("data-image")


                # nutrients
                nutrients = soup.find_all("tbody")
                
                nutrient = nutrients[0].getText()
                
                
                # link
                links = soup.find_all("meta", property="og:url")
                link = links[0].get("content")
                
                lst_name.append(name[0])
                lst_brand.append(brand[0])
                lst_link.append(link)
                lst_price.append(price[0])
                lst_weight.append(weight[0])
                lst_price_gr.append(price_gr[0])
                lst_nutrients.append(nutrient)
                lst_photos.append(photo_link)

            except: 
                
                print("error for: ", l)
        
        
    return [lst_name, lst_brand, lst_link, lst_price, lst_weight, lst_price_gr, lst_nutrients, lst_photos]


**Testing function**

In [152]:
links=['https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854']

product_1 = scraping_details(links, *skip_link)
product_1

[['Mandeln mit Zimt & Zucker'],
 ['Vista Portuguese'],
 ['https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854'],
 ['  7,90\xa0€ '],
 ['  Inhalt:  125\xa0g '],
 [' 6,32\xa0€ / 100\xa0g '],
 ['  Brennwert 2.022 kJ(442 kcal)   Fett 26,0 g   davon gesättigte Fettsäuren 2,0 g   Kohlenhydrate 59,0 g   davon Zucker 52,0 g   Eiweiß 11,0 g   Salz 0,00 g  '],
 ['https://foodist.imgix.net/media/image/5602132652727_0.jpg?auto=compress%2Cformat&h=60&q=20&w=60']]

In [153]:
two_products = scraping_details(links=['https://www.foodist.de/n.a-nature-addicts-fruchtsticks-himbeere-10895', 
                                       'https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854'])
two_products

[['Fruchtsticks Himbeere', 'Mandeln mit Zimt & Zucker'],
 ['N.A! Nature Addicts', 'Vista Portuguese'],
 ['https://www.foodist.de/n.a-nature-addicts-fruchtsticks-himbeere-10895',
  'https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854'],
 ['  0,90\xa0€ ', '  7,90\xa0€ '],
 ['  Inhalt:  16\xa0g ', '  Inhalt:  125\xa0g '],
 [' 5,63\xa0€ / 100\xa0g ', ' 6,32\xa0€ / 100\xa0g '],
 ['  Brennwert 1.415 kJ(334 kcal)   Fett 1,2 g   davon gesättigte Fettsäuren 0,3 g   Kohlenhydrate 75,0 g   davon Zucker 69,0 g   Ballaststoffe 8,1 g   Eiweiß 18,0 g   Salz 0,10 g  ',
  '  Brennwert 2.022 kJ(442 kcal)   Fett 26,0 g   davon gesättigte Fettsäuren 2,0 g   Kohlenhydrate 59,0 g   davon Zucker 52,0 g   Eiweiß 11,0 g   Salz 0,00 g  '],
 ['https://foodist.imgix.net/media/image/3609200010757_0.jpg?auto=compress%2Cformat&h=60&q=20&w=60',
  'https://foodist.imgix.net/media/image/5602132652727_0.jpg?auto=compress%2Cformat&h=60&q=20&w=60']]

In [154]:
skip_link=['https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685']
links=['https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685',
                  'https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854', 
                   'https://www.foodist.de/n.a-nature-addicts-fruchtsticks-himbeere-10895']

test = scraping_details(links, skip_link)
test[0]

error for:  https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685


['Mandeln mit Zimt & Zucker', 'Fruchtsticks Himbeere']

In [155]:
test

[['Mandeln mit Zimt & Zucker', 'Fruchtsticks Himbeere'],
 ['Vista Portuguese', 'N.A! Nature Addicts'],
 ['https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854',
  'https://www.foodist.de/n.a-nature-addicts-fruchtsticks-himbeere-10895'],
 ['  7,90\xa0€ ', '  0,90\xa0€ '],
 ['  Inhalt:  125\xa0g ', '  Inhalt:  16\xa0g '],
 [' 6,32\xa0€ / 100\xa0g ', ' 5,63\xa0€ / 100\xa0g '],
 ['  Brennwert 2.022 kJ(442 kcal)   Fett 26,0 g   davon gesättigte Fettsäuren 2,0 g   Kohlenhydrate 59,0 g   davon Zucker 52,0 g   Eiweiß 11,0 g   Salz 0,00 g  ',
  '  Brennwert 1.415 kJ(334 kcal)   Fett 1,2 g   davon gesättigte Fettsäuren 0,3 g   Kohlenhydrate 75,0 g   davon Zucker 69,0 g   Ballaststoffe 8,1 g   Eiweiß 18,0 g   Salz 0,10 g  '],
 ['https://foodist.imgix.net/media/image/5602132652727_0.jpg?auto=compress%2Cformat&h=60&q=20&w=60',
  'https://foodist.imgix.net/media/image/3609200010757_0.jpg?auto=compress%2Cformat&h=60&q=20&w=60']]

#### Function to create dataframe

In [158]:
def creating_df(links, *list_):

    result = scraping_details(links, *list_)

    details = pd.DataFrame(
        {"name": result[0],
        "brand": result[1],
        "links": result[2],
        "price": result[3],
        "weight": result[4],
        "price_gr": result[5],
        "nutritients": result[6],
        "photo_link": result[7]}
        )
        
    return details

In [159]:
skip_link=['https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685']
links=['https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685',
                  'https://www.foodist.de/vista-portuguese-mandeln-mit-zimt-zucker-10854', 
                   'https://www.foodist.de/n.a-nature-addicts-fruchtsticks-himbeere-10895']
creating_df(links, *skip_link)

,name,brand,links,price,weight,price_gr,nutritients,photo_link
0,Mandeln mit Zimt & Zucker,Vista Portuguese,https://www.foodist.de/vista-portuguese-mandel...,"7,90 €",Inhalt: 125 g,"6,32 € / 100 g","Brennwert 2.022 kJ(442 kcal) Fett 26,0 g ...",https://foodist.imgix.net/media/image/56021326...
1,Fruchtsticks Himbeere,N.A! Nature Addicts,https://www.foodist.de/n.a-nature-addicts-fruc...,"0,90 €",Inhalt: 16 g,"5,63 € / 100 g","Brennwert 1.415 kJ(334 kcal) Fett 1,2 g ...",https://foodist.imgix.net/media/image/36092000...


**Checking which links don't work**

In [111]:
test = scraping_details(links=product_overview['links'])

error for:  https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685
error for:  https://www.foodist.de/foodist-feinschmecker-adventskalender-2021-von-thomas-buehner-11069
error for:  https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092
error for:  https://www.foodist.de/craft-beer-adventskalender
error for:  https://www.foodist.de/gaia-bienenwachstuch-starterset-jungle-3-teilig-10074
error for:  https://www.foodist.de/foodist-bio-spice-your-pumpkin-pie-gewuerzmischung-by-marry-kotter-11340
error for:  https://www.foodist.de/foodist-foodist-snack-bites-probierpaket-10-teilig-8465
error for:  https://www.foodist.de/foodist-bio-pralinen-adventskalender-2021-11249
error for:  https://www.foodist.de/foodist-bio-schokoladen-weihnachtsmann-sparset-4-teilig-11704
error for:  https://www.foodist.de/the-beginnings-cookies-doppelpack-mit-orange-cranberry-11565
error for:  https://www.foodist.de/foodist-scharfe-erdnuesse-10915
error for:  https://w

**Getting product details**

In [160]:
links=product_overview['links']
skip_link= [
    'https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685',
    'https://www.foodist.de/foodist-feinschmecker-adventskalender-2021-von-thomas-buehner-11069',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/craft-beer-adventskalender',
    'https://www.foodist.de/gaia-bienenwachstuch-starterset-jungle-3-teilig-10074',
    'https://www.foodist.de/foodist-bio-spice-your-pumpkin-pie-gewuerzmischung-by-marry-kotter-11340',
    'https://www.foodist.de/foodist-foodist-snack-bites-probierpaket-10-teilig-8465',
    'https://www.foodist.de/foodist-bio-pralinen-adventskalender-2021-11249',
    'https://www.foodist.de/foodist-bio-schokoladen-weihnachtsmann-sparset-4-teilig-11704',
    'https://www.foodist.de/the-beginnings-cookies-doppelpack-mit-orange-cranberry-11565',
    'https://www.foodist.de/foodist-scharfe-erdnuesse-10915',
    'https://www.foodist.de/foodloose-bio-fruchtgummi-mango-10870',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/veganer-adventskalender',
    'https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685',
    'https://www.foodist.de/foodist-feinschmecker-adventskalender-2021-von-thomas-buehner-11069',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/craft-beer-adventskalender',
    'https://www.foodist.de/gaia-bienenwachstuch-starterset-jungle-3-teilig-10074',
    'https://www.foodist.de/foodist-bio-spice-your-pumpkin-pie-gewuerzmischung-by-marry-kotter-11340',
    'https://www.foodist.de/foodist-foodist-snack-bites-probierpaket-10-teilig-8465',
    'https://www.foodist.de/foodist-bio-pralinen-adventskalender-2021-11249',
    'https://www.foodist.de/foodist-bio-schokoladen-weihnachtsmann-sparset-4-teilig-11704',
    'https://www.foodist.de/the-beginnings-cookies-doppelpack-mit-orange-cranberry-11565',
    'https://www.foodist.de/foodist-scharfe-erdnuesse-10915',
    'https://www.foodist.de/foodloose-bio-fruchtgummi-mango-10870',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/veganer-adventskalender',
    'https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685',
    'https://www.foodist.de/foodist-feinschmecker-adventskalender-2021-von-thomas-buehner-11069',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/craft-beer-adventskalender',
    'https://www.foodist.de/gaia-bienenwachstuch-starterset-jungle-3-teilig-10074',
    'https://www.foodist.de/foodist-bio-spice-your-pumpkin-pie-gewuerzmischung-by-marry-kotter-11340',
    'https://www.foodist.de/foodist-foodist-snack-bites-probierpaket-10-teilig-8465',
    'https://www.foodist.de/foodist-bio-pralinen-adventskalender-2021-11249',
    'https://www.foodist.de/foodist-bio-schokoladen-weihnachtsmann-sparset-4-teilig-11704',
    'https://www.foodist.de/the-beginnings-cookies-doppelpack-mit-orange-cranberry-11565',
    'https://www.foodist.de/foodist-scharfe-erdnuesse-10915',
    'https://www.foodist.de/foodloose-bio-fruchtgummi-mango-10870',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/veganer-adventskalender',
    'https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685',
    'https://www.foodist.de/foodist-feinschmecker-adventskalender-2021-von-thomas-buehner-11069',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/craft-beer-adventskalender',
    'https://www.foodist.de/gaia-bienenwachstuch-starterset-jungle-3-teilig-10074',
    'https://www.foodist.de/foodist-bio-spice-your-pumpkin-pie-gewuerzmischung-by-marry-kotter-11340',
    'https://www.foodist.de/foodist-foodist-snack-bites-probierpaket-10-teilig-8465',
    'https://www.foodist.de/foodist-bio-pralinen-adventskalender-2021-11249',
    'https://www.foodist.de/foodist-bio-schokoladen-weihnachtsmann-sparset-4-teilig-11704',
    'https://www.foodist.de/the-beginnings-cookies-doppelpack-mit-orange-cranberry-11565',
    'https://www.foodist.de/foodist-scharfe-erdnuesse-10915',
    'https://www.foodist.de/foodloose-bio-fruchtgummi-mango-10870',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/veganer-adventskalender',
    'https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685',
    'https://www.foodist.de/foodist-feinschmecker-adventskalender-2021-von-thomas-buehner-11069',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/craft-beer-adventskalender',
    'https://www.foodist.de/gaia-bienenwachstuch-starterset-jungle-3-teilig-10074',
    'https://www.foodist.de/foodist-bio-spice-your-pumpkin-pie-gewuerzmischung-by-marry-kotter-11340',
    'https://www.foodist.de/foodist-foodist-snack-bites-probierpaket-10-teilig-8465',
    'https://www.foodist.de/foodist-bio-pralinen-adventskalender-2021-11249',
    'https://www.foodist.de/foodist-bio-schokoladen-weihnachtsmann-sparset-4-teilig-11704',
    'https://www.foodist.de/the-beginnings-cookies-doppelpack-mit-orange-cranberry-11565',
    'https://www.foodist.de/foodist-scharfe-erdnuesse-10915',
    'https://www.foodist.de/foodloose-bio-fruchtgummi-mango-10870',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/veganer-adventskalender',
    'https://www.foodist.de/foodist-bier-braukasten-set-pils-zum-selbermachen-diy-kit-7685',
    'https://www.foodist.de/foodist-feinschmecker-adventskalender-2021-von-thomas-buehner-11069',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/craft-beer-adventskalender',
    'https://www.foodist.de/gaia-bienenwachstuch-starterset-jungle-3-teilig-10074',
    'https://www.foodist.de/foodist-bio-spice-your-pumpkin-pie-gewuerzmischung-by-marry-kotter-11340',
    'https://www.foodist.de/foodist-foodist-snack-bites-probierpaket-10-teilig-8465',
    'https://www.foodist.de/foodist-bio-pralinen-adventskalender-2021-11249',
    'https://www.foodist.de/foodist-bio-schokoladen-weihnachtsmann-sparset-4-teilig-11704',
    'https://www.foodist.de/the-beginnings-cookies-doppelpack-mit-orange-cranberry-11565',
    'https://www.foodist.de/foodist-scharfe-erdnuesse-10915',
    'https://www.foodist.de/foodloose-bio-fruchtgummi-mango-10870',
    'https://www.foodist.de/foodist-foodist-snacks-adventskalender-2021-11092',
    'https://www.foodist.de/veganer-adventskalender'
    ]

In [161]:
products_foodist = creating_df(links, *skip_link)

In [162]:
products_foodist.head(10)

,name,brand,links,price,weight,price_gr,nutritients,photo_link
0,BIO Chiasamen 1kg,Foodist,https://www.foodist.de/foodist-bio-chiasamen-1...,"8,95 €",Inhalt: 1000 g,"0,90 € / 100 g","Brennwert 1.901 kJ(454 kcal) Fett 31,0 g ...",https://foodist.imgix.net/media/image/42604537...
1,Xylit Bonbons Orange,Birkengold,https://www.foodist.de/birkengold-xylit-bonbon...,"2,50 €",Inhalt: 30 g,"8,33 € / 100 g","Brennwert 1.072 kJ(257 kcal) Fett 1,8 g ...",https://foodist.imgix.net/media/image/91200488...
2,Schokolade & Spielzeug Weihnachten,PLAYin Choc,https://www.foodist.de/playin-choc-schokolade-...,"4,00 €",Inhalt: 20 g,"20,00 € / 100 g","Brennwert 2.609 kJ(630 kcal) Fett 51,6 g ...",https://foodist.imgix.net/media/image/50605485...
3,"Waffelteig gefüllt mit Erdnussbutter ""Nutisfac...",Paddies,https://www.foodist.de/paddies-waffelteig-gefu...,"1,99 €",Inhalt: 70 g,"2,84 € / 100 g","Brennwert 1.742 kJ(418 kcal) Fett 25,0 g ...",https://foodist.imgix.net/media/image/91201027...
4,"BIO Marshmallows mit Vanillegeschmack ""Marilyn...",Not Guilty,https://www.foodist.de/not-guilty-bio-marshmal...,"1,90 €",Inhalt: 80 g,"2,38 € / 100 g","Brennwert 1.386 kJ(326 kcal) Fett 0,0 g ...",https://foodist.imgix.net/media/image/36634580...
5,Kekse mit Schokostückchen,Heldenbrot,https://www.foodist.de/heldenbrot-kekse-mit-sc...,"1,90 €",Inhalt: 25 g,"7,60 € / 100 g","Brennwert 1.917 kJ(457 kcal) Fett 19,0 g ...",https://foodist.imgix.net/media/image/42605459...
6,"BIO Riegel Banane, Rote Traube und Aronia 4er-...",Freche Freunde,https://www.foodist.de/freche-freunde-bio-rieg...,"1,75 €",Inhalt: 92 g,"1,90 € / 100 g","Brennwert 1.611 kJ(383 kcal) Fett 13,0 g ...",https://foodist.imgix.net/media/image/42602491...
7,KakaoWUMMS - Gebrannte Kakaobohnen mit Chili,PURmacherei,https://www.foodist.de/purmacherei-kakaowumms-...,"3,49 €",Inhalt: 50 g,"6,98 € / 100 g","Brennwert 2.184 kJ(527 kcal) Fett 35,0 g ...",https://foodist.imgix.net/media/image/42700004...
8,"BIO Getrocknete Kirschen, knusprig",Supersec,https://www.foodist.de/supersec-bio-getrocknet...,"4,85 €",Inhalt: 140 g,"3,46 € / 100 g","Brennwert 1.380 kJ(325 kcal) Fett 0,5 g ...",https://foodist.imgix.net/media/image/54250331...
9,"Kleine Freuden Sparset, 2-teilig",Foodist,https://www.foodist.de/foodist-kleine-freuden-...,"29,98 €",Inhalt: 1 Stck.,"29,98 € / 1 Stck.","Brennwert 2.564 kJ(614 kcal) Fett 48,0 g ...",https://foodist.imgix.net/media/image/Bundle_S...


In [163]:
products_foodist.shape

(462, 8)

In [164]:
products_foodist.to_csv('df_foodist.csv')